In [2]:
from selenium import webdriver
from urllib.parse import quote_plus
from urllib.request import urlopen
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import urllib.request
import os

In [6]:
# def save_images(images, save_path):
#     for index, image in enumerate(images[:200]):  # images[:크롤링하고 싶은 사진 개수]
#         src = image.get_attribute('src')
#         t = urlopen(src).read()
#         file = open(os.path.join(save_path, str(index + 1) + ".jpg"), "wb")
#         file.write(t)
#         print("img save " + save_path + str(index + 1) + ".jpg")
 
 
def create_folder_if_not_exists(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)
 
 
def make_url(search_term):
    # 네이버 이미지 검색
    base_url = 'https://search.naver.com/search.naver?where=image&section=image&query='
    # CCL 상업적 이용 가능 옵션
    # end_url = '&res_fr=0&res_to=0&sm=tab_opt&color=&ccl=2' \
    #           '&nso=so%3Ar%2Ca%3Aall%2Cp%3Aall&recent=0&datetype=0&startdate=0&enddate=0&gif=0&optStr=&nso_open=1'
    return base_url + quote_plus(search_term) #+ end_url
 
 
def crawl_images(search_term):
    # URL 생성
    url = make_url(search_term)
 
    # chrome 브라우저 열기
    browser = webdriver.Chrome('chromedriver')
    browser.implicitly_wait(3)  # 브라우저를 오픈할 때 시간간격을 준다.
    print(f'explore url: {url}')
    browser.get(url)
#    browser.maximize_window()
    
     #페이지 스크롤 다운
    body= browser.find_element(By.CSS_SELECTOR, "body")
    
    for i in range(20):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1) #delay 주기
    
#     # 스크롤 끝까지 내려 검색결과 더 보기
#     SCROLL_PAUSE_TIME = 10
    
#     # 무한 스크롤 처리
#     # 스크롤 내리기 전 위치
#     before_h = browser.execute_script("return window.scrollY")
    
#     s_count = 0
#     # 무한스크롤
#     while True:
#         print(f'scrolling count : {s_count}')
#         #맨 아래로 스크롤 내리기
#         browser.find_element(By.CSS_SELECTOR, "body.wrap-new.api_animation.tabsch.tabsch_image").send_keys(Keys.END)
#         # 스크롤 사이 페이지 로딩 시간
#         time.sleep(SCROLL_PAUSE_TIME)
#         # 스크롤 후 높이
#         after_h = browser.execute_script("return window.scrollY")
        
#         s_count = s_count +1
#         if after_h == before_h:
#             break
#         before_h = after_h

    
    imgs = browser.find_elements(By.CLASS_NAME, "_image")
    print(f'total imgs : {len(imgs)}')
    # 저장 경로 설정
    dir_path = "../Data/carbon_image_crawling/" + search_term + "/" # 이미지 저장 경로 (리눅스)
    create_folder_if_not_exists(dir_path)
    
    count = 0  # 이미지 번호
    for img in imgs:  # 각 이미지 순회
        if count > 1600:  # 테스트용으로 이미지 5개만 다운로드 한다. 검색결과 페이지를 모두 긁으려면 이 조건문을 없앤다.
             break
        try:
            img.click()  # 검색결과 이미지를 클릭하여
            time.sleep(1)  # 상세보기가 다운될 동안 대기
            imgUrl  = browser.find_element(By.XPATH,
                '//*[@id="main_pack"]/section[2]/div/div[2]/div/div/div[1]/div[1]/div/div/div[1]/div[1]/img').get_attribute(
                "src")  # 구글 이미지검색결과 페이지의 내부구조가 바뀌면 여길 수정해야 한다.
            
            urllib.request.urlretrieve(imgUrl, dir_path + "/" + search_term +'_'+ str(count).zfill(3) + ".jpg")
            print(f'count: {count}, imgUrl: {imgUrl}')
            
        except Exception as e:
            print(e)
        count = count + 1
            
    # 마무리
    print(search_term + " 저장 성공")
    browser.close()

In [8]:
if __name__ == '__main__':
    crawl_images('우유팩 접기')

explore url: https://search.naver.com/search.naver?where=image&section=image&query=%EC%9A%B0%EC%9C%A0%ED%8C%A9+%EC%A0%91%EA%B8%B0
total imgs : 349
count: 0, imgUrl: https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAxOTExMDlfNjIg%2FMDAxNTczMjM1MzE2Mjkx.qA0MgAUbeSk52Lyzz10r9JtdmdIsVx5D1SSjjhnZrBMg.tHF8sRHml8ayrr0htkzoL2-eDrHE3IIvL4J1ns6q1xEg.JPEG.snow443%2F%25BF%25EC%25C0%25AF%25B0%25FB%25BB%25F3%25C0%25DA%25BD%25E61.jpg&type=sc960_832
count: 1, imgUrl: https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyMDEyMTJfMjMy%2FMDAxNjA3Nzc5ODgyMzAz.rTHHdXj1LVVSAip77tiCnAyGDlO65ujGHSaRS_L9rF8g.2bFgXXueYioMdS1uhmHkFe4x4YJA9qXk_IT0I8F2-L8g.JPEG.didi1225%2FKakaoTalk_20201212_215231836_23.jpg&type=sc960_832
count: 2, imgUrl: https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2F20160807_186%2Fdalae4848_14705330169700a82A_JPEG%2FIMG_6143.JPG&type=sc960_832
count: 3, imgUrl: https://search.pstatic.net/common/?src=http%3A%2F%2Fblogf

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pyautogui
import os

# 검색어 입력.
keyword = keyword = pyautogui.prompt("검색어를 입력해주세요.")

# 저장 폴더 중복 검사 
pd_exists =  os.path.exists(f'{keyword}')

# 조건에 따른 저장 폴더 생성법 ①
if True == pd_exists:
    pass
else:
    os.mkdir(f'{keyword}')

# 조건에 따른 저장 폴더 생성법 ②
# if not pd_exists:
    # os.mkdir(f'{keyword}')

# 크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager

# 브라우저 자동 꺼짐 방지하기
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메세지 안보이게 하기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 크롬드라이버매니저를 통해서 최신 크롬드라이버를 자동으로 설치하고,
# service 객체를 만들어 변수에 저장.
service = Service(executable_path=ChromeDriverManager().install())

# 크롬 열고, 화면 최대화
browser = webdriver.Chrome(service=service, options=chrome_options)
browser.maximize_window()

# 검색어 입력했을때, 네이버 이미지 탭 결과 리스트창
browser.get(f"https://search.naver.com/search.naver?where=image&query={keyword}")

# 웹페이지가 로딩 될 때까지 10초 기다림. 뜨면 바로 10초 안채워도 넘어감.
browser.implicitly_wait(10)

# 무한 스크롤 처리
# 스크롤 내리기 전 위치
before_h = browser.execute_script("return window.scrollY")

# 무한스크롤
while True:
    #맨 아래로 스크롤 내리기
    browser.find_element(By.CSS_SELECTOR, "body.wrap-new.api_animation.tabsch.tabsch_image").send_keys(Keys.END)
    # 스크롤 사이 페이지 로딩 시간
    time.sleep(3)
    # 스크롤 후 높이
    after_h = browser.execute_script("return window.scrollY")

    if after_h == before_h:
        break
    before_h = after_h